In [1]:
import os
from os.path import join

original_dir = 'D:\\Images\\Data\\CatsVsDogs'
train_dir = join(original_dir, 'train')
train_dog_dir = join(train_dir, 'dog')
train_cat_dir = join(train_dir, 'cat')

test_dir = join(original_dir, 'test')

validation_dir = join(original_dir, 'validation')
validation_dog_dir = join(validation_dir, 'dog')
validation_cat_dir = join(validation_dir, 'cat')

print(f'TRAINING cats images: {len(os.listdir(train_cat_dir))}')
print(f'TRAINING dogs images: {len(os.listdir(train_dog_dir))}')
print(f'TOTAL:\t{len(os.listdir(train_cat_dir)) + len(os.listdir(train_dog_dir))}')

print(f'VALIDATION cats images: {len(os.listdir(validation_cat_dir))}')
print(f'VALIDATION dogs images: {len(os.listdir(validation_dog_dir))}')
print(f'TOTAL:\t{len(os.listdir(validation_cat_dir)) + len(os.listdir(validation_dog_dir))}')

print(f'TEST images: {len(os.listdir(test_dir))}')

TRAINING cats images: 10000
TRAINING dogs images: 10000
TOTAL:	20000
VALIDATION cats images: 2500
VALIDATION dogs images: 2500
TOTAL:	5000
TEST images: 12500


In [2]:
from tensorflow.keras import layers, models

model = models.Sequential()
model.add(layers.Conv2D(32,
                        (3, 3),
                        activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________

In [3]:
from tensorflow.keras import optimizers

model.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
simple_datagen = ImageDataGenerator(rescale=1./255)

train_generator = simple_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=100,
        class_mode='binary')

validation_generator = simple_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=100,
        class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [5]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=200,
      epochs=15,
      validation_data=validation_generator,
      validation_steps=50)

Epoch 1/15
200/200 [==============================] - 47s 235ms/step - loss: 0.6576 - acc: 0.5997 - val_loss: 0.6025 - val_acc: 0.6812
Epoch 2/15
200/200 [==============================] - 42s 209ms/step - loss: 0.5698 - acc: 0.7029 - val_loss: 0.5476 - val_acc: 0.7176
Epoch 3/15
200/200 [==============================] - 42s 210ms/step - loss: 0.5163 - acc: 0.7470 - val_loss: 0.4934 - val_acc: 0.7616
Epoch 4/15
200/200 [==============================] - 42s 210ms/step - loss: 0.4809 - acc: 0.7656 - val_loss: 0.4924 - val_acc: 0.7640
Epoch 5/15
200/200 [==============================] - 42s 208ms/step - loss: 0.4529 - acc: 0.7867 - val_loss: 0.4721 - val_acc: 0.7676
Epoch 6/15
200/200 [==============================] - 42s 208ms/step - loss: 0.4322 - acc: 0.7996 - val_loss: 0.4495 - val_acc: 0.7946
Epoch 7/15
200/200 [==============================] - 42s 209ms/step - loss: 0.4122 - acc: 0.8133 - val_loss: 0.4133 - val_acc: 0.8142
Epoch 8/15
200/200 [==============================] - 4

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()